In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle
import os, sys

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)
os.chdir("..")

2.2.3
3.4.2


In [3]:
os.getcwd()

'/Users/benschaefer/Desktop/epa141-model/final assignment'

In [4]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator, Scenario
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

region_name = "zutphen"
RES_PATH = f"./saved_runs/directed_search/search_worst_case_{region_name}.pkl"
nfe = 2000
formulation = 3

In [5]:
with open(f"./saved_runs/reference_cases/worst_case_{region_name}.pkl", "rb") as file:
    reference_vals = pickle.load(file)

reference_dict = reference_vals.to_dict()
reference_dict.pop("scenario")
reference_dict.pop("model")
scenario1 = Scenario(f"Worst case {region_name}", **reference_dict)
print(reference_dict)
# scenario2 = Scenario("Stable dikes", )


{'A.0_ID flood wave shape': 123, 'A.1_Bmax': 34.61895861529783, 'A.1_Brate': 1.0, 'A.1_pfail': 0.8532106085979978, 'A.2_Bmax': 308.93875651967915, 'A.2_Brate': 1.5, 'A.2_pfail': 0.9830011389751809, 'A.3_Bmax': 250.97169787883655, 'A.3_Brate': 10.0, 'A.3_pfail': 0.12023358916896845, 'A.4_Bmax': 267.1380819724575, 'A.4_Brate': 1.5, 'A.4_pfail': 0.18678120667023174, 'A.5_Bmax': 189.1637565416625, 'A.5_Brate': 1.5, 'A.5_pfail': 0.48080911947870375, 'discount rate 0': 2.5, 'EWS_DaysToThreat': 0, 'policy': 'Do Nothing', 'A.1_Expected Annual Damage': 462062.29055261996, 'A.1_Dike Investment Costs': 0.0, 'A.1_Expected Number of Deaths': 0.00029021037014673, 'A.2_Expected Annual Damage': 4664833.377016683, 'A.2_Dike Investment Costs': 0.0, 'A.2_Expected Number of Deaths': 0.004000028562813652, 'A.3_Expected Annual Damage': 857916852.9141109, 'A.3_Dike Investment Costs': 0.0, 'A.3_Expected Number of Deaths': 1.2329670561284671, 'A.4_Expected Annual Damage': 0.0, 'A.4_Dike Investment Costs': 0.0,

In [6]:
ema_logging.log_to_stderr(ema_logging.INFO)
os.chdir("/Users/benschaefer/Desktop/epa141-model/final assignment")
# print(os.getcwd())
# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(7)
for lever in dike_model.levers:
    pass
    # print(lever.lower_bound, lever.upper_bound)

[IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False), IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False), IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False), IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False), IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False), IntegerParameter('EWS_DaysToThreat', 0, 1, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False), IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False), IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False), IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIn

In [7]:
with MultiprocessingEvaluator(dike_model) as evaluator:
            results = evaluator.optimize(
                nfe=nfe,
                searchover='levers',
                reference=scenario1,
                epsilons=[0.1] * len(dike_model.outcomes),
                # convergence=convergence_metrics,
                # constraints=constraints
            )

[MainProcess/INFO] pool started with 8 workers
100%|██████████████████████████████████████| 2000/2000 [04:45<00:00,  7.02it/s]
[MainProcess/INFO] optimization completed, found 80 solutions
[MainProcess/INFO] terminating pool


In [8]:
results

,0_RfR 0,1_RfR 0,2_RfR 0,3_RfR 0,4_RfR 0,EWS_DaysToThreat,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Expected Number of Deaths
0,0,1,1,1,0,0,0,2,7,7,5,0.000000e+00,1.279783e+08,369700000.0,0.000000
1,1,0,0,1,0,0,0,2,9,6,6,2.032449e+05,1.362876e+08,205800000.0,0.000107
2,1,0,1,0,0,1,0,0,8,4,3,6.391676e+06,7.733176e+07,115300000.0,0.001557
3,0,0,0,0,0,0,1,3,8,7,6,1.134017e+06,1.723580e+08,0.0,0.001822
4,0,0,0,0,0,0,0,0,10,7,0,9.538286e+07,5.847667e+07,0.0,0.074860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,1,0,0,0,1,0,7,7,5,3.235443e+06,1.178548e+08,30700000.0,0.002747
76,0,0,1,1,0,0,0,0,6,4,2,1.333319e+07,6.867222e+07,151900000.0,0.009354
77,0,0,1,0,0,1,0,0,6,2,3,3.082446e+07,6.908240e+07,30700000.0,0.006880
78,0,0,0,0,0,1,0,0,5,8,4,3.801079e+07,7.828443e+07,0.0,0.019281


In [9]:
with open(RES_PATH, "wb") as file:
    pickle.dump(results, file)